In [1]:
pip install librosa scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import librosa
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import joblib

In [29]:
# 사용자 폴더 구조 기반 경로 정의
base_dir = "./2501ml_data"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
label_dir = os.path.join(base_dir, "label")
train_label_path = os.path.join(label_dir, "train_label.txt")
test_label_path = os.path.join(label_dir, "test_label.txt")

# 폴더, 라벨 경로 확인
(train_dir, test_dir, train_label_path, test_label_path)

('./2501ml_data/train',
 './2501ml_data/test',
 './2501ml_data/label/train_label.txt',
 './2501ml_data/label/test_label.txt')

In [31]:
## 라벨 불러오는 함수 부분

In [38]:
def load_labels(label_file_path):
    labels = {}
    with open(label_file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 4:
                fname = parts[1]              # ex: KDF_T_0001.wav
                label = parts[4].lower()      # Real/Fake → 소문자로
                labels[fname] = label
    return labels


In [39]:
train_labels = load_labels(train_label_path)

for i, (k, v) in enumerate(train_labels.items()):
    print(k, v)
    if i > 10:
        break


KDF_T_0001.wav real
KDF_T_0002.wav real
KDF_T_0003.wav real
KDF_T_0004.wav real
KDF_T_0005.wav real
KDF_T_0006.wav real
KDF_T_0007.wav real
KDF_T_0008.wav real
KDF_T_0009.wav real
KDF_T_0010.wav real
KDF_T_0011.wav real
KDF_T_0012.wav real


In [32]:
## MFCC 39차원 벡터 추출 함수

In [40]:
def extract_mfcc_39(filepath):
    try:
        y, sr = librosa.load(filepath, sr=16000)  # 16kHz로 로딩
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # (13, time)
        delta = librosa.feature.delta(mfcc)                # Δ
        delta2 = librosa.feature.delta(mfcc, order=2)      # ΔΔ
        mfcc_39 = np.vstack([mfcc, delta, delta2])         # (39, time)
        return np.mean(mfcc_39, axis=1)                    # 평균 내서 (39,) 벡터
    except Exception as e:
        print(f"[오류] {filepath}: {e}")
        return None


In [ ]:
## 사용 예시

In [41]:
vec = extract_mfcc_39(os.path.join(train_dir, "KDF_T_0001.wav"))
print(vec.shape)  # → (39,)
print(vec[:5])    # 일부 벡터 값 출력


(39,)
[-362.77438   122.49903    26.545534   25.488754    6.769586]


In [42]:
real_vecs = []
fake_vecs = []

for fname, label in train_labels.items():
    path = os.path.join(train_dir, fname)
    vec = extract_mfcc_39(path)
    if vec is None:
        continue
    if label == "real":
        real_vecs.append(vec)
    elif label == "fake":
        fake_vecs.append(vec)


In [43]:
print(f"✅ real 벡터 수: {len(real_vecs)}")
print(f"✅ fake 벡터 수: {len(fake_vecs)}")


✅ real 벡터 수: 2000
✅ fake 벡터 수: 2000


In [49]:
gmm_real = GaussianMixture(n_components=8, covariance_type='diag', random_state=42)
gmm_real.fit(real_vecs)

gmm_fake = GaussianMixture(n_components=8, covariance_type='diag', random_state=42)
gmm_fake.fit(fake_vecs)


GaussianMixture(covariance_type='diag', n_components=8, random_state=42)

In [52]:
for i, (fname, true_label) in enumerate(test_labels.items()):
    path = os.path.join(test_dir, fname)
    vec = extract_mfcc_39(path)
    if vec is None:
        continue
    log_real = gmm_real.score([vec])
    log_fake = gmm_fake.score([vec])
    pred = "real" if log_real > log_fake else "fake"

    print(f"{fname} → pred: {pred}, true: {true_label.lower()}, log_real: {log_real:.2f}, log_fake: {log_fake:.2f}")
    
    if i >= 9:
        break


KDF_E_1004.wav → pred: real, true: real, log_real: -9.76, log_fake: -37.48
KDF_E_0510.wav → pred: real, true: real, log_real: -30.14, log_fake: -36.57
KDF_E_1538.wav → pred: real, true: real, log_real: -16.05, log_fake: -50.85
KDF_E_0370.wav → pred: real, true: real, log_real: -4.82, log_fake: -39.12
KDF_E_1048.wav → pred: real, true: real, log_real: -22.33, log_fake: -45.70
KDF_E_0607.wav → pred: real, true: real, log_real: -26.58, log_fake: -53.47
KDF_E_1163.wav → pred: real, true: real, log_real: -8.44, log_fake: -29.02
KDF_E_1147.wav → pred: real, true: real, log_real: 3.46, log_fake: -24.94
KDF_E_1828.wav → pred: real, true: real, log_real: -19.26, log_fake: -29.74
KDF_E_0421.wav → pred: real, true: real, log_real: -21.30, log_fake: -39.65


In [53]:
correct = 0
total = 0
results = []

for fname, true_label in test_labels.items():
    path = os.path.join(test_dir, fname)
    vec = extract_mfcc_39(path)
    if vec is None:
        continue
    log_real = gmm_real.score([vec])
    log_fake = gmm_fake.score([vec])
    pred = "real" if log_real > log_fake else "fake"
    
    # 정답 비교
    if pred == true_label.lower():
        correct += 1
    total += 1
    
    results.append(f"{fname} {pred}\n")


In [55]:
accuracy = correct / total
accuracy_percent = accuracy * 100
print(f"🎯 정확도: {accuracy_percent:.2f}%  ({correct}/{total})")

🎯 정확도: 76.30%  (1526/2000)
